In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_columns',999)

In [3]:
def file_head(file):
    with open(file) as csv:
        tabular = []
        for x in range(10):
            tabular.append(csv.readline().strip().split(','))
        return pd.DataFrame(tabular[1:],columns=tabular[0])

In [34]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
!cd data & dir


 Volume in drive H has no label.
 Volume Serial Number is 1425-AB82

 Directory of H:\Kaggle\elo-merchant-category-recommendation\data

12/18/2018  03:22 PM    <DIR>          .
12/18/2018  03:22 PM    <DIR>          ..
12/18/2018  03:08 PM            16,917 Data_Dictionary.xlsx
11/23/2018  09:08 PM     2,845,920,484 historical_transactions.csv
11/23/2018  09:08 PM        50,040,976 merchants.csv
11/23/2018  09:08 PM       190,246,145 new_merchant_transactions.csv
11/23/2018  09:08 PM         2,225,229 sample_submission.csv
11/23/2018  09:08 PM         3,708,740 test.csv
11/23/2018  09:08 PM         8,383,651 train.csv
               7 File(s)  3,100,542,142 bytes
               2 Dir(s)   8,709,644,288 bytes free


merchant_id - Unique merchant identifier<br>
merchant_group_id - Merchant group (anonymized )<br>
merchant_category_id - Unique identifier for merchant category (anonymized )<br>
subsector_id - Merchant category group (anonymized )<br>
numerical_1 - anonymized measure <br>
numerical_2 - anonymized measure <br>
category_1 - anonymized category <br>
most_recent_sales_range - Range of revenue (monetary units) in last active month --> A > B > C > D > E <br>
most_recent_purchases_range - Range of quantity of transactions in last active month --> A > B > C > D > E <br>
avg_sales_lag3 - Monthly average of revenue in last 3 months divided by revenue in last active month <br>
avg_purchases_lag3 - Monthly average of transactions in last 3 months divided by transactions in last active month <br>
active_months_lag3 - Quantity of active months within last 3 months <br>
avg_sales_lag6 - Monthly average of revenue in last 6 months divided by revenue in last active month <br>
avg_purchases_lag6 - Monthly average of transactions in last 6 months divided by transactions in last active month <br>
active_months_lag6 - Quantity of active months within last 6 months <br>
avg_sales_lag12 - Monthly average of revenue in last 12 months divided by revenue in last active month <br>
avg_purchases_lag12 - Monthly average of transactions in last 12 months divided by transactions in last active month <br>
active_months_lag12 - Quantity of active months within last 12 months <br>
category_4 - anonymized category <br>
city_id	City - identifier (anonymized ) <br>
state_id - State identifier (anonymized ) <br>
category_2 - anonymized category<br>

In [28]:
dtps = {'city_id':np.int8,'installments':np.int8,'merchant_category_id':np.int16,'month_lag':np.int8,'state_id':np.int16,'subsector_id':np.int16}
hist_trans = pd.read_csv('data\\historical_transactions.csv',dtype=dtps,parse_dates=['purchase_date'])
merchants = pd.read_csv('data\\merchants.csv')
merch_trans = pd.read_csv('data\\new_merchant_transactions.csv',parse_dates=['purchase_date'])


In [6]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

In [7]:
train = read_data('data/train.csv')
test = read_data('data/test.csv')

target = train['target']
del train['target']

In [19]:
hist_trans['month_diff'] = ((datetime.datetime.today() - hist_trans.purchase_date).dt.days)//30
hist_trans.month_diff += hist_trans.month_lag

In [29]:
merch_trans['month_diff'] = ((datetime.datetime.today() - merch_trans.purchase_date).dt.days)//30
merch_trans.month_diff += merch_trans.month_lag

In [31]:
hist_trans.category_1.replace({'Y':1, 'N':0},inplace=True)
hist_trans.authorized_flag.replace({'Y':1, 'N':0},inplace=True)
merch_trans.category_1.replace({'Y':1, 'N':0},inplace=True)
merch_trans.authorized_flag.replace({'Y':1, 'N':0},inplace=True)

In [32]:
hist_trans = pd.get_dummies(hist_trans,columns=['category_2', 'category_3'])
merch_trans = pd.get_dummies(merch_trans,columns=['category_2', 'category_3'])

In [35]:
historical_transactions = reduce_mem_usage(hist_trans)
new_transactions = reduce_mem_usage(merch_trans)

Mem. usage decreased to 1249.37 Mb (30.8% reduction)
Mem. usage decreased to 86.12 Mb (58.9% reduction)


In [54]:
agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)

In [55]:
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]

In [47]:
auth_mean

,authorized_flag_mean
card_id,
C_ID_00007093c1,0.765101
C_ID_0001238066,0.975610
C_ID_0001506ef0,0.939394
C_ID_0001793786,0.875000
C_ID_000183fdda,0.951389
C_ID_00024e244b,0.757143
C_ID_0002709b5a,0.917808
C_ID_00027503e2,0.571429
C_ID_000298032a,0.933333


In [56]:
auth_mean.reset_index(inplace=True)

In [58]:
authorized_trans = historical_transactions[historical_transactions.authorized_flag == 1]
historical_trans = historical_transactions[historical_transactions.authorized_flag == 0]

In [62]:
at = authorized_trans.copy()
ht = historical_trans.copy()

In [66]:
at['purchase_month'] = authorized_trans.purchase_date.dt.month
ht['purchase_month'] = historical_trans.purchase_date.dt.month
new_transactions['purchase_month'] = new_transactions.purchase_date.dt.month

In [65]:
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
    'category_1': ['sum', 'mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'subsector_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_month': ['mean', 'max', 'min', 'std'],
    'purchase_date': [np.ptp, 'min', 'max'],
    'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean']
    }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

In [67]:
history = aggregate_transactions(ht)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]

KeyError: "Column 'month_diff' does not exist!"